### **Estimación de frecuencia y elementos mayoritarios**

La **estimación de frecuencias** en flujos de datos (data streams) es un reto central en sistemas que procesan información de forma continua y en tiempo real. Cuando los elementos llegan uno a uno, potencialmente sin límite de longitud, resulta inviable almacenarlos todos para contar con exactitud. En su lugar, buscamos métodos que, en tiempo y espacio sublineales, permitan obtener estimaciones suficientemente precisas.

**Elemento mayoritario**

Se define **elemento mayoritario** a aquel que aparece más de $\lfloor n/2\rfloor$ veces en un flujo de $n$ elementos. Un enfoque clásico y elegante para hallarlo en tiempo $O(n)$ y espacio $O(1)$ es el algoritmo de *Boyer–Moore majority vote*. La idea es mantener un contador y un candidato, de modo que tras procesar todo el flujo, el candidato sea el único posible:

1. **Fase de selección de candidato**  
   Inicialmente, $count=0$. Por cada elemento $x$:
   - Si $count=0$, asignamos $candidate\leftarrow x$ y $count\leftarrow1$.
   - En caso contrario, si $x=candidate$ incrementamos $count\leftarrow count+1$, y si $x\neq candidate$ decrementamos $count\leftarrow count-1$.

2. **Fase de verificación (opcional)**  
   Para asegurar que el candidato realmente supera $\lfloor n/2\rfloor$, se recorre de nuevo el flujo contando sus ocurrencias y comprobando
   $$\#\{\,x : x=candidate\}\;>\;\frac{n}{2}.$$

El candidato final satisface:

$$
\sum_{i=1}^n \bigl[\,a_i = \mathit{candidate}\bigr] \;-\;\sum_{i=1}^n \bigl[\,a_i \neq \mathit{candidate}\bigr]\;>\;0
$$
lo que garantiza su mayoría cuando existe un único elemento que cumple la condición. 

**Heavy hitters**

Cuando extendemos el umbral de $\frac{n}{2}$ a un valor arbitrario $\varphi n$ (con $0<\varphi<1$), surgen los **heavy hitters**: los elementos cuya frecuencia excede $\varphi n$. El algoritmo de Misra–Gries (o _frequent_) mantiene a lo sumo $k=\lfloor 1/\varphi\rfloor$ contadores y procede así:

1. Se mantiene un conjunto $S$ de hasta $k$ pares $(elemento, contador)$.
2. Al procesar $x$:
   - Si $x\in S$, incrementamos su contador.
   - Si $|S|<k$, añadimos $(x,1)$ a $S$.
   - Si $|S|=k$ y $x\notin S$, decrementamos $1$ a todos los contadores y eliminamos aquellos que lleguen a cero.

Al final, los elementos con contador positivo en $S$ son candidatos, que tras una segunda pasada pueden verificarse contando sus ocurrencias reales. Este enfoque usa espacio $O(1/\varphi)$ y tiempo amortizado $O(1)$ por elemento, ofreciendo una cota $(\varphi n)$ de error en la estimación de frecuencias.



### **Count–Min Sketch: estructura y funcionamiento**

El **Count–Min Sketch** (CMS) se basa en la combinación de varias estimaciones sesgadas con la operación de mínimo para obtener una cota superior ajustada de la frecuencia real. A continuación se describen con detalle sus componentes teóricos, variantes de actualización, garantías probabilísticas y aspectos prácticos de implementación.

**Hashing y requerimientos de independencia**

Cada función de hash $h_i$ debe pertenecer a una familia universal (o al menos 2-independiente) para garantizar que, para cualquier par de elementos distintos $x\neq y$, 
$$
\Pr[h_i(x)=h_i(y)] \;=\; \frac{1}{w}.
$$
Con independencia par a par basta para el análisis de error:

- **Colisiones esperadas**: para un elemento $x$, el número de inserciones de otros elementos que caen en la misma celda $(i,h_i(x))$ tiene valor esperado
  $$
  \mathbb{E}\bigl[C[i,h_i(x)] - f(x)\bigr]
  = \sum_{y\neq x}\Pr[h_i(y)=h_i(x)]
  = \frac{N - f(x)}{w}.
  $$
- **Acotación de cola**: usando la desigualdad de Markov,
  $$
  \Pr\bigl[C[i,h_i(x)] - f(x) \ge \varepsilon N\bigr]
  \;\le\;
  \frac{\mathbb{E}[C[i,h_i(x)] - f(x)]}{\varepsilon N}
  = \frac{N/w}{\varepsilon N}
  = \frac{1}{e},
  $$
  pues elegimos $w = \lceil e/\varepsilon\rceil$.

Repetir esta estimación en $d$ filas independientes y tomar el mínimo asegura:
$$
\Pr\bigl[\min_{i\in[d]}C[i,h_i(x)] - f(x) \ge \varepsilon N\bigr]
\;\le\;\Bigl(\tfrac1e\Bigr)^d
\;\le\;\delta,
$$
si $d \ge \lceil \ln(1/\delta)\rceil$.


**Actualizaciones conservadoras**

La **actualización conservadora** (conservative update) es una mejora empírica que reduce el sesgo al evitar incrementar todas las entradas:

1. Para cada $i\in\{1,\dots,d\}$, calcular primero  
   $\widehat f_i = C[i,h_i(x)]$.
2. Determinar $\widehat f_{\min} = \min_i \widehat f_i$.
3. Solo incrementar aquellas celdas cuyo valor sea igual a $\widehat f_{\min}$:
   $$
   \forall\,i:\; C[i,h_i(x)] 
   \;+\!=\; 
   \begin{cases}
   1, & \text{si }C[i,h_i(x)] = \widehat f_{\min},\\
   0, & \text{si }C[i,h_i(x)] > \widehat f_{\min}.
   \end{cases}
   $$
Con ello, se evita propagar conteos excesivos en todas las filas, reduciendo la sobrestimación promedio, aunque sigue preservando el mismo orden de complejidad:

- **Tiempo por update**: $O(d)$.
- **Error máximo**: aún acotado por $\varepsilon N$ con probabilidad $1-\delta$.


**Fusión de sketches (merge/composability)**

Los CMS son **linealmente composables**: dadas dos matrices $C^A$ y $C^B$ construidas con los mismos parámetros $(d,w,h_1,\dots,h_d)$ sobre flujos disjuntos, la fusión se obtiene por suma elemento a elemento:

$$
C[i,j] \;\leftarrow\; C^A[i,j] \;+\; C^B[i,j],
\quad
\forall\,i\in[1,d],\;j\in[0,w-1].
$$

La estimación tras mezclar satisface las mismas garantías estadísticas para el flujo combinado, pues

$$
\widehat f_{\text{global}}(x)
= \min_i\bigl(C^A[i,h_i(x)] + C^B[i,h_i(x)]\bigr).
$$

Esto es fundamental en entornos distribuidos: cada nodo procesa su parte, envía su CMS al agregador y se realiza un único `merge`, obteniendo un CMS global con costes $O(d\,w)$.

**Extensión a actualizaciones ponderadas y decrementos**

- **Actualizaciones ponderadas**: si cada elemento $x$ viene con un peso $w_x$, basta reemplazar el incremento unitario por
  $$
  C[i,h_i(x)] \;+\!=\; w_x,
  $$
  manteniendo intacta la garantía de error, ahora en proporción a $\sum w_x$.

- **Decrementos**: en aplicaciones de ventanas deslizantes (sliding windows), al retirar un evento con peso $w_x$ se podría hacer
  $$
  C[i,h_i(x)] \;-\!=\; w_x
  \quad\text{(limitando a 0 para evitar negativos).}
  $$
  Sin embargo, esto rompe las garantías pues las colisiones no son reversibles. En su lugar, se suelen emplear **sketches con contadores con tiempo** o esquemas de **exponentially decaying windows** que mantienen versiones antiguas y reconstruyen el CMS periódicamente.


**Operaciones avanzadas: producto interno y estimación de joins**

Dado un CMS para flujo $A$ con matriz $C^A$ y otro para flujo $B$ con $C^B$, se puede estimar el **producto interno** (join size):

$$
\sum_{x} f_A(x)\,f_B(x)
$$

mediante

$$
\widehat{J} \;=\; \sum_{j=0}^{w-1}\;\min\bigl(C^A[i,j],\,C^B[i,j]\bigr)
$$

para cada fila $i$, y tomando la mínima de esas sumas. Sirve, por ejemplo, para estimar el tamaño de la intersección de conjuntos en bases de datos.

**Implementación de bajo nivel y consideraciones prácticas**

1. **Tipos de contadores**  
   - Usar enteros de 32 bits con **saturación** (al desbordar, quedan en $\mathtt{UINT32\_MAX}$) para evitar wrap-around.
   - Opción de contadores de 16 bits si la frecuencia máxima es baja.

2. **Elección de hashes**  
   - MurmurHash3 o CityHash con semillas distintas para cada fila.
   - Asegurarse de independencia práctica: claves y semillas no deben solaparse.

3. **Localidad de memoria**  
   - Almacenar la matriz como un bloque contiguo:   `{uint32_t table[d][w];}`  
     para mejorar la cacheabilidad, accediendo a $\texttt{table[i*w + idx]}$.

4. **Paralelismo**  
   - En arquitecturas multihilo, cada hilo puede tener su CMS local y luego hacer un **reducer** que sume las matrices.
   - Alternativamente, usar atomics en la tabla global, aunque con más contención.



#### **Casos de uso**

**Identificación de "restless sleepers"**

En aplicaciones de monitoreo del sueño, cada movimiento de usuario genera un evento. El objetivo es mantener actualizada, en tiempo real, la lista de los $k$ usuarios con mayor número de eventos (los _k restless sleepers_). Una estrategia práctica:

1. Instanciar un CMS para contar eventos por usuario, con parámetros $(\varepsilon,\delta)$ adecuados al volumen.
2. Cada vez que llega un evento para el usuario $u$, ejecutar `CMS.update(u)`.
3. Mantener un **heap mínimo** de tamaño $k$ (heap) que almacene usuarios y sus frecuencias estimadas.
4. Tras cada actualización, si la frecuencia estimada de $u$ supera el mínimo del heap, sustituirlo por $u$.

Gracias al espacio sublineal del CMS y a la eficiencia del heap ($O(\log k)$ por inserción), es factible procesar millones de identificadores con recursos controlados.

**Similitud distribucional de palabras en grandes corpus**

En procesamiento de lenguaje natural, la **similitud distribucional** de dos palabras $w_1,w_2$ se basa en contar sus contextos coocurrentes:
$$
\text{PMI}(w_1,w_2)
=\log\frac{f(w_1,w_2)\,N}{f(w_1)\,f(w_2)},
$$
donde $f(\,\cdot\,)$ indica frecuencia y $N$ es el número total de pares. Con un CMS:

1. Para cada par $(w,contexto)$ en el flujo de texto, llamar a `CMS.update("pair:"+w+"#"+contexto)`.
2. Para estimar $f(w_1,w_2)$, usar `CMS.estimate("pair:"+w_1+"#"+w_2)`; para $f(w_i)$, otro CMS dedicado a frecuencias marginales.
3. Calcular $\widehat{\text{PMI}}(w_1,w_2)$ sustituyendo las estimaciones en la fórmula.

Este método escala a vocabularios de decenas de millones de términos, reduciendo drásticamente la memoria comparado con tablas hash exactas.

**Detección de ataques DDoS en tiempo real**

En redes, un ataque DDoS genera ráfagas de paquetes desde múltiples orígenes. Se puede:

1. Contar paquetes por dirección IP fuente mediante CMS.
2. Definir un umbral de frecuencia (por ejemplo, $\varphi N$) para detectar IPs sospechosas.
3. Identificar rápidamente heavy hitters y activar defensas automáticas.

Al combinar CMS con estructuras de heavy hitters (Misra–Gries), se consigue un sistema ligero y reactivo.

#### **Error vs. espacio en Count–Min Sketch**

El diseño de un CMS implica un compromiso entre precisión y memoria. La tabla comparativa muestra la relación:


| Parámetro | Dimensión                         | Garantía de error                                                     |
|-----------|-----------------------------------|-----------------------------------------------------------------------|
| $w$       | $\lceil e / \varepsilon \rceil$   | $\lvert \widehat{f}(x) - f(x) \rvert \le \varepsilon N$              |
| $d$       | $\lceil \ln(1 / \delta) \rceil$   | $P(\text{error} \ge \varepsilon N) \le \delta$                      |
| Total     | $d \times w$                      | $O\left( \frac{1}{\varepsilon} \ln\left( \frac{1}{\delta} \right) \right)$ |

- Si elegimos un $\varepsilon$ menor, aumentamos $w\propto1/\varepsilon$, lo que incrementa el espacio linealmente.
- Para reducir la probabilidad de fallo $\delta$, crece $d\propto\ln(1/\delta)$, impactando el tiempo por operación.

Este análisis permite adaptar el CMS a las limitaciones de memoria y al nivel de confianza requerido.



#### **Implementación sencilla en Python**

A continuación se presenta una versión minimalista de Count–Min Sketch:



In [ ]:
import mmh3
from array import array
from math import ceil, e, log
from typing import Union

class CountMinSketch:
    """
    Estructura probabilística Count–Min Sketch para estimar frecuencias.

    Parámetros
    ----------
    epsilon : float
        Fracción de error permitido (error aditivo máximo = epsilon * total_eventos).
    delta : float
        Probabilidad máxima de que el error exceda epsilon * total_eventos.
    conservative : bool, opcional
        Si es True, usa actualización conservadora para reducir la sobreestimación.
    """
    def __init__(
        self,
        epsilon: float,
        delta: float,
        conservative: bool = False,
    ):
        # Validar rangos de parámetros
        if not (0 < epsilon < 1):
            raise ValueError(f"epsilon debe estar en (0,1), pero vino {epsilon}")
        if not (0 < delta < 1):
            raise ValueError(f"delta debe estar en (0,1), pero vino {delta}")

        # Guardar parámetros
        self.epsilon = epsilon
        self.delta = delta
        self.conservative = conservative

        # Dimensiones: ancho w y número de filas d
        self.w = ceil(e / epsilon)
        self.d = ceil(log(1 / delta))
        self.total = 0                   # contador total de todos los eventos

        # Matriz d×w inicializada a ceros
        self.table = [array('L', [0] * self.w) for _ in range(self.d)]

        # Semillas de 32 bits para cada función de hash
        self.seeds = [((i * 0xFBA4C795 + 1) & 0xFFFFFFFF) for i in range(self.d)]

    def update(self, key: Union[str, bytes], count: int = 1) -> None:
        """
        Incrementa la cuenta de `key` en `count` (debe ser >=1).
        """
        if count < 1:
            raise ValueError(f"count debe ser >=1, pero vino {count}")

        # Calcular los índices en cada fila
        indices = [mmh3.hash(key, seed) % self.w for seed in self.seeds]

        if self.conservative:
            # Actualización conservadora: sólo incrementa las celdas con el valor mínimo actual
            valores = [self.table[i][idx] for i, idx in enumerate(indices)]
            minimo = min(valores)
            for i, idx in enumerate(indices):
                if self.table[i][idx] == minimo:
                    self.table[i][idx] += count
        else:
            # Actualización normal: incrementa todas las filas
            for i, idx in enumerate(indices):
                self.table[i][idx] += count

        self.total += count

    def estimate(self, key: Union[str, bytes]) -> int:
        """
        Devuelve la estimación de frecuencia de `key`.
        """
        return min(
            self.table[i][mmh3.hash(key, seed) % self.w]
            for i, seed in enumerate(self.seeds)
        )

    def merge(self, other: 'CountMinSketch') -> 'CountMinSketch':
        """
        Fusiona otro CountMinSketch en este (modifica en lugar).

        Ambos sketches deben tener los mismos d, w y seeds.
        """
        if not isinstance(other, CountMinSketch):
            raise TypeError("Sólo se puede fusionar con otro CountMinSketch")
        if (self.w, self.d, self.seeds) != (other.w, other.d, other.seeds):
            raise ValueError("Dimensiones o semillas de hash diferentes")

        for i in range(self.d):
            for j in range(self.w):
                self.table[i][j] += other.table[i][j]
        self.total += other.total
        return self

    def __len__(self) -> int:
        """
        Devuelve el conteo total de todos los elementos procesados.
        """
        return self.total

    def __repr__(self) -> str:
        return (
            f"CountMinSketch(epsilon={self.epsilon:.4f}, "
            f"delta={self.delta:.4e}, total={self.total}, "
            f"d={self.d}, w={self.w})"
        )



**Ejemplos**

In [ ]:
# Crear un CMS con error máximo del 1% y probabilidad de fallo 0.1%
cms = CountMinSketch(epsilon=0.01, delta=0.001)
# Crear otro CMS idéntico para demostrar merge
cms2 = CountMinSketch(epsilon=0.01, delta=0.001, conservative=True)

# Actualizar frecuencias
palabras = ["gato", "perro", "gato", "ratón", "perro", "gato"]
for p in palabras:
    cms.update(p)
    cms2.update(p, count=2)  # en cms2 usamos actualización ponderada

# Estimar frecuencias
for p in {"gato", "perro", "ratón"}:
    print(f"{p}: estimación cms={cms.estimate(p)}, cms2={cms2.estimate(p)}")

# Fusionar cms2 en cms
cms.merge(cms2)
print("\nTras fusionar cms2 en cms:")
for p in {"gato", "perro", "ratón"}:
    print(f"{p}: estimación combinada={cms.estimate(p)}")

# Mostrar contador total
print(f"\nTotal de eventos procesados: {len(cms)}")


#### **Consultas por rangos con Count–Min Sketch**

Sea un dominio ordenado de enteros $[0, U-1]$. Queremos responder a la consulta  
$$
F(L,R)\;=\;\sum_{x=L}^{R} f(x),
$$  
donde $f(x)$ es la frecuencia (o peso) del ítem $x$. El CMS estándar no soporta directamente sumas sobre rangos—pero podemos hacerlo con **descomposición diádica** y mantener un CMS por cada "nivel" de potencias de dos.

**Niveles y bloques diádicos**

- Definimos $k = \lceil\log_{2} U\rceil$.  
- Para cada nivel $i=0,1,\dots,k$, los bloques diádicos de longitud $2^i$ son  
  $$
  B_{i,j} = \bigl[j\cdot 2^i,\;(j+1)\cdot2^i -1\bigr],\quad j=0,1,\dots,\lfloor U/2^i\rfloor.
  $$
- Hay a lo sumo $\tfrac{U}{2^i}+1$ bloques en el nivel $i$ (muchos de ellos vacíos si $U$ no es múltiplo).

Cada bloque $B_{i,j}$ representamos su contador con la clave de hash  
```text
key = f"{i}:{j}"
```
en un CMS independiente $\mathrm{CMS}_i$.

**Fase de actualización ("update")**

**Objetivo:** cuando llega un elemento $x$, debemos incrementar *todos* los bloques diádicos que lo contienen, es decir, uno por cada nivel.

```python
def update(x, count=1):
    for i in range(0, k+1):
        # bloque diádico que contiene x en nivel i
        j = x >> i           # equivale a floor(x / 2^i)
        CMS[i].update(f"{i}:{j}", count)
```

- **Complejidad**  
  - Coste de hash y escritura: $O(d)$ por cada CMS,  
  - hay $k+1 = O(\log U)$ niveles,  
  - ⇒ **Actualización en** $O(d\,\log U)$.

- **Memoria**  
  - Cada CMS_i ocupa $d\times w$ contadores,  
  - Total: $(k+1)\,d\,w = O\bigl(d\,w\,\log U\bigr)$.

**Fase de consulta de rango ("query")**

Para obtener $\widehat F(L,R)$:

1. **Descomposición diádica**: calculamos un conjunto de pares $\mathcal B\subseteq\{0,\dots,k\}\times\mathbb N$ que cubren exactamente $[L,R]$ sin solaparse.  
2. **Suma de estimaciones**:
   $$
   \widehat F(L,R)
   = \sum_{(i,j)\in\mathcal B} \mathrm{CMS}_i.\mathrm{estimate}(i \mathbin{:} j)
   $$

donde $i \mathbin{:} j$ representa una codificación única del intervalo que corresponde al bloque $(i, j)$, por ejemplo mediante la cadena de caracteres $\texttt{"i:j"}$, usada internamente por CMS como clave.

**¿Por qué $O(\log U)$ bloques?**

- Cada descomposición diádica elige siempre el bloque más grande alineado con $L$ que quepa dentro de $[L,R]$.  
- Tras consumir ese bloque, el nuevo inicio se mueve al final del mismo y volvemos a elegir el bloque más grande posible.  
- Este "greedy" garantiza a lo sumo $2\log_2 U$ bloques:
  - **Parte izquierda**: a lo sumo $\log_2 U$ bloques (uno por cada bit en la representación de $L$).  
  - **Parte derecha**: análogo para la longitud $(R-L+1)$.  

Por ejemplo, para $[3,14]$ con $U\ge16$:
```
[3,14] = [3,3] ∪ [4,7] ∪ [8,15]∩[8,14]
        = B_{0,3} ∪ B_{2,1} ∪ (B_{3,1} truncated)
```
que corresponden a 3 bloques, mucho menos que $U$.

**Pseudocódigo completo**

```python
def dyadic_intervals(L: int, R: int) -> list[tuple[int,int]]:
    """Devuelve lista de (nivel i, bloque j) que cubren [L,R]."""
    intervals = []
    while L <= R:
        # Máxima potencia de dos que divide a L
        max_block = L & -L
        # Longitud restante
        rem = R - L + 1
        # Ajustar block al tamaño que quepa en rem
        block = max_block if max_block <= rem else 1 << (rem.bit_length() - 1)
        lvl = block.bit_length() - 1
        j = L >> lvl
        intervals.append((lvl, j))
        L += block
    return intervals

def range_query(L: int, R: int) -> int:
    """Estimación de frecuencia en [L,R] usando k+1 CountMinSketch."""
    total = 0
    for i, j in dyadic_intervals(L, R):
        key = f"{i}:{j}"
        total += CMS[i].estimate(key)
    return total
```

- **Coste**  
  - Construir $\mathcal B$: $O(\log U)$.  
  - Por cada bloque, una consulta CMS en $O(d)$.  
  - => **Complejidad**: $O(d\,\log U) = O(\ln(1/\delta)\,\log U)$.

**Garantías de error para rango**

Cada bloque incursiona un error **aditivo** $\le \varepsilon N$ con probabilidad $1-\delta$. Al sumar $|\mathcal B|$ bloques:

- El error global $\le |\mathcal B|\cdot\varepsilon N \le 2\varepsilon\,N\log_2U$.  
- Para mantener un **error relativo** pequeño, conviene elegir un $\varepsilon'$ tal que  
  $$
  \varepsilon' \;=\;\frac{\varepsilon}{2\log_2U},
  $$
  y crear los CMS con ese parámetro para que la suma de errores siga $\le \varepsilon N$.

**Variantes y optimizaciones**

1. **Segment tree de sketches**  
   - En lugar de niveles diádicos fijos, construir un árbol binario completo sobre $[0,U-1]$, con un CMS en cada nodo.  
   - La descomposición y consulta usan el mismo principio de árbol, pero ocupa $O(U)$ nodos: más memoria, pero consultas en $O(d\log U)$ igual.

2. **Descomposición dinámica**  
   - Si el flujo usa sólo un subconjunto de $[0, U-1]$, podemos llevar un mapeo dinámico de bloques activos y mantener CMS solo para ellos, ahorrando espacio.

3. **Ajuste de parámetros por nivel**  
   - Podríamos dedicar más ancho $w$ a niveles donde esperamos mayor densidad de elementos (niveles bajos), y menos a los altos.


### **Estimación de cardinalidad y hyperloglog**

En aplicaciones de **big data**, bases de datos distribuidas y procesado de flujos en tiempo real, es común la necesidad de **contar elementos distintos** (cardinalidad) en un conjunto de datos muy grande o en un flujo continuo. El método exacto por ejemplo, mantener un _hash set_ de todos los valores vistos, es inviable cuando:

- El volumen de datos supera la capacidad de memoria disponible.  
- Se requiere una respuesta en tiempo real, con latencias muy bajas.  
- Se procesan datos distribuidos en varias máquinas y luego debe agregarse el resultado.

Los algoritmos de **conteo probabilístico** resuelven este problema entregando una estimación de la cardinalidad con un error controlado y usando **espacio sublineal**. Entre ellos, la familia **hyperloglog (HLL)** se ha consolidado como uno de los más precisos y eficientes, ofreciendo un **error relativo** del orden de $1\%$ usando apenas unos pocos kilobytes de memoria.




#### **Conteo de elementos distintos en bases de datos**

**Enfoque exacto**

El método tradicional para contar distintos en SQL es:

```sql
SELECT COUNT(DISTINCT columna) FROM tabla;
```

Este enfoque requiere escanear la tabla completa y almacenar en memoria o en disco la lista de valores únicos (o su índice), con un coste en espacio y tiempo de $O(n)$. Para tablas de decenas o centenas de millones de filas, el rendimiento y la escalabilidad son problemáticos.


En entornos de **streaming**, los datos llegan de forma continua y no pueden almacenarse en su totalidad. Además, el requisito de latencia impide:

- Realizar dos pasadas sobre los datos.  
- Almacenar todas las claves vistas.  

Por ello, se necesitan estructuras que:

1. **Procesen cada elemento** en tiempo amortizado muy bajo ($O(1)$ o $O(\log\log n)$).  
2. **Usen espacio sublineal** respecto al número de elementos distintos.  
3. **Entreguen estimaciones** con error relativo bajo y ajustable.

#### **Diseño incremental de HyperLogLog**

La evolución de HyperLogLog parte del algoritmo clásico de Flajolet–Martin y pasa por dos etapas intermedias antes de llegar a la fórmula definitiva basada en media armónica. A continuación se presenta cada paso con texto fluido y las ecuaciones principales.

**Probabilistic counting (Flajolet–Martin)**

En el enfoque original se emplea una única función de hash $h$ que mapea cada elemento $x$ a una palabra de $L$ bits, interpretada como una secuencia de ceros y unos. A partir de ese hash, se define
$$
\rho(h(x)) = \min\{\,i \ge 1 : \text{el $i$-ésimo bit de }h(x)\text{ es }1\},
$$
es decir, la posición del primer uno (contando ceros iniciales). Se mantiene un solo registro
$$
R = \max_{x\in\text{stream}} \rho\bigl(h(x)\bigr),
$$
y la cardinalidad aproximada del conjunto es
$$
\hat{n} = 2^R.
$$
Aunque simple, este método adolece de una varianza muy elevada ($\sigma\approx n$) y un error relativo cercano al 100 %, dado que depende de un único máximo.

**Promedio estocástico**

Para atenuar la varianza se introducen $k$ funciones de hash independientes $h_1,\dots,h_k$ y se calcula un registro $R_i$ para cada una. La estimación se define como la media aritmética de sus potencias:
$$
\bar{Z}
= \frac{1}{k}\sum_{i=1}^k 2^{R_i}.
$$
La varianza de $\bar{Z}$ cae a $O(n^2/k)$, de modo que el error relativo pasa a ser $O(1/\sqrt{k})$. Sin embargo, este enfoque multiplica por $k$ tanto el espacio requerido como el coste de computar cada elemento.

**LogLog**

LogLog consigue un efecto similar con un único vector de $m$ registros, donde $m=2^b$. Se aprovechan los primeros $b$ bits del hash para escoger un bucket $j\in\{0,\dots,m-1\}$ y el resto de bits para calcular $\rho$. Para cada bucket $j$ se mantiene
$$
R_j \;=\; \max_{x:\,\text{bucket}(x)=j} \rho\bigl(\text{suffix}(h(x))\bigr).
$$
La estimación global se formula como
$$
\hat{n}
= \alpha_m \,m \,2^{\frac{1}{m}\sum_{j=0}^{m-1}R_j},
$$
donde $\alpha_m$ es una constante de corrección de sesgo. Así, la varianza baja a $O(n^2/m)$ y el error relativo a $O(1/\sqrt{m})$, pero la media de potencias deja un sesgo residual.

**HyperLogLog: media armónica y correcciones**

HyperLogLog sustituye la media de potencias por una media armónica que reduce drásticamente el sesgo. Definiendo

$$
Z \;=\;\Bigl(\sum_{j=0}^{m-1}2^{-R_j}\Bigr)^{-1},
$$

la estimación principal es

$$
\hat{n}
=\frac{\alpha_m\,m^2}{\sum_{j=0}^{m-1}2^{-R_j}},
$$

donde

$$
\alpha_m
=
\begin{cases}
0.673 & m=16,\\
0.697 & m=32,\\
0.709 & m=64,\\
\frac{0.7213}{1 + 1.079/m} & m\ge128.
\end{cases}
$$

Además, para mejorar la precisión en los extremos se aplican dos correcciones:

1. **Rango bajo** ($\hat n \le 2.5\,m$): usar _linear counting_
   $$
     \hat{n}_{\mathrm{LC}}
     = m\;\ln\!\Bigl(\tfrac{m}{V}\Bigr),
   $$
   donde $V$ es el número de registros con $R_j=0$.

2. **Rango alto** ($\hat n > 2^L/30$, con $L$ bits de hash):
   $$
     \hat{n}_{\mathrm{HR}}
     = -2^{L}\;\ln\!\Bigl(1 - \tfrac{\hat{n}}{2^{L}}\Bigr).
   $$

Con estas mejoras, HyperLogLog alcanza un error relativo teórico de
$$
\sigma \approx \frac{1.04}{\sqrt{m}},
$$
logrando, por ejemplo con $m=2^{14}=16384$, una precisión cercana al 0.8 %.

#### **Implementación de HyperLogLog en Python**

A continuación presentamos una **implementación simplificada** de HyperLogLog en Python, que incluye:

- Cálculo de buckets por los primeros $b$ bits.  
- Registro del máximo de ceros iniciales.  
- Estimación y correcciones de rango bajo.  
- Fusión (_merge_) de dos estructuras HLL.

In [ ]:
import mmh3        # MurmurHash3
import math
from typing import Callable, Optional

class HyperLogLog:
    """
    HyperLogLog para estimación de cardinalidad de un stream de elementos.
    Soporta:
      - Parámetro b: log2(m) buckets
      - Corrección de rango bajo (linear counting)
      - Corrección de rango alto (opcional)
      - Fusión de múltiples HLL
    """

    def __init__(
        self,
        b: int,
        hash_fn: Optional[Callable[[bytes], int]] = None,
        register_type: type = list
    ):
        """
        :param b: número de bits para indexar (m = 2^b buckets).
        :param hash_fn: función que tome bytes y devuelva un entero (por defecto mmh3.hash64).
        :param register_type: tipo de contenedor para los registros; por defecto list.
        """
        if b < 4 or b > 16:
            raise ValueError("b debe estar entre 4 y 16 para garantizar precisión y eficiencia")
        self.b = b
        self.m = 1 << b
        self.registers = register_type([0]) * self.m

        # Selección de función de hash de 64 bits (más uniforme en volúmenes grandes)
        self._hash = hash_fn or (lambda v: mmh3.hash64(v, signed=False)[0])

        # Constante α_m
        if   self.m == 16:   self.alpha = 0.673
        elif self.m == 32:   self.alpha = 0.697
        elif self.m == 64:   self.alpha = 0.709
        else:                self.alpha = 0.7213 / (1 + 1.079 / self.m)

        # Número de bits para ρ()
        self._w_bits = 64 - self.b

    def _rho(self, w: int) -> int:
        """
        Cuenta ceros iniciales en w (de longitud _w_bits) + 1.
        Usa bit_length() para velocidad.
        """
        if w == 0:
            return self._w_bits + 1
        # posición del bit más alto (1-based)
        l = w.bit_length()
        # ceros a la izquierda: (_w_bits - l)
        return (self._w_bits - l) + 1

    def add(self, value: str) -> None:
        """
        Inserta un elemento (string) en el HLL.
        Convierte a bytes UTF-8 antes de hashear.
        """
        x = self._hash(value.encode('utf-8'))
        idx = x >> self._w_bits        # primeros b bits
        w   = x & ((1 << self._w_bits) - 1)
        self.registers[idx] = max(self.registers[idx], self._rho(w))

    def estimate(self) -> float:
        """
        Retorna la estimación corregida de cardinalidad.
        Aplica:
          1. Estimación cruda E
          2. Linear counting si E es pequeño
          3. Corrección de rango alto si E es muy grande
        """
        inv_sum = sum(2.0 ** -r for r in self.registers)
        E = (self.alpha * self.m * self.m) / inv_sum

        # Corrección rango bajo
        if E <= 2.5 * self.m:
            V = self.registers.count(0)
            if V:
                E = self.m * math.log(self.m / V)

        # Corrección rango alto (para cardinalidades cercanas al espacio de hash)
        elif E > (1/30) * (1 << 64):
            E = - (1 << 64) * math.log(1 - E / (1 << 64))

        return E

    def merge(self, other: 'HyperLogLog') -> 'HyperLogLog':
        """
        Fusiona self con otro HLL (de mismo b) y retorna uno nuevo.
        """
        if self.b != other.b:
            raise ValueError("No es posible fusionar HLLs con distinto parámetro b")
        h = HyperLogLog(self.b, hash_fn=self._hash, register_type=type(self.registers))
        h.registers = [max(r1, r2) for r1, r2 in zip(self.registers, other.registers)]
        return h

    def __or__(self, other: 'HyperLogLog') -> 'HyperLogLog':
        """Permite usar operador '|' como alias de merge()."""
        return self.merge(other)

    def __len__(self) -> int:
        """Longitud aproximada de elementos insertados."""
        return int(self.estimate())

    def __repr__(self) -> str:
        return f"<HyperLogLog b={self.b} m={self.m} est={self.estimate():.2f}>"


**Ejemplos**

In [ ]:
def ejemplo_básico():
    # 1) Creamos un HLL con b = 10 -> m = 2^10 = 1024 buckets
    hll = HyperLogLog(b=10)

    # 2) Insertamos 10000 elementos únicos (como strings)
    for i in range(10000):
        hll.add(f"user_{i}")

    # 3) Estimación de cardinalidad
    print("Estimación cruda:    ", hll.estimate())
    print("Estimación entera:   ", len(hll))
    print("Representación .repr:", hll)

def ejemplo_merge():
    # Creamos dos HLL independientes
    h1 = HyperLogLog(b=10)
    h2 = HyperLogLog(b=10)

    # h1 recibe los elementos 0–4999, h2 los 5000–9999
    for i in range(5000):
        h1.add(f"item_{i}")
    for i in range(5000, 10000):
        h2.add(f"item_{i}")

    # Cada uno estima ~5000
    print("h1 ≈", len(h1), " h2 ≈", len(h2))

    #  - Fusionamos con el método merge()
    h_merged = h1.merge(h2)
    print("Merge():", len(h_merged))       # ≈10000

    #  - O usando el operador |
    h_union = h1 | h2
    print("Operator | :", len(h_union))   # ≈10000

def ejemplo_hash_personalizado():
    # Podemos pasar nuestra propia función de hash
    import hashlib

    def sha1_hash(v: bytes) -> int:
        # Tomamos los 8 bytes más bajos de SHA1 para obtener un entero de 64 bits
        h = hashlib.sha1(v).digest()[-8:]
        return int.from_bytes(h, "big")

    # Usamos SHA1 en lugar de mmh3
    h_sha1 = HyperLogLog(b=12, hash_fn=sha1_hash)

    # Insertamos algunos valores repetidos
    datos = ["apple", "banana", "cherry", "apple", "banana"] * 100
    for fruta in datos:
        h_sha1.add(fruta)

    print("Únicos esperados: 3")
    print("Estimación SHA1 :", len(h_sha1))  # ≈3

# Ejecutar todos los ejemplos de una vez
print("Ejemplo básico")
ejemplo_básico()
print("\nEjemplo de merge")
ejemplo_merge()
print("\nEjemplo con hash personalizado")
ejemplo_hash_personalizado()


#### **Caso de uso: "Catching Worms" con HyperLogLog**

En un entorno de **detección de gusanos de red** (network worms), el volumen de paquetes maliciosos puede ser inmenso y los recursos de memoria y CPU escasos. HyperLogLog (HLL) encaja perfectamente como estructura de conteo aproximado de **hosts únicos**, gracias a su bajo consumo de memoria y alta velocidad en la inserción de cada fingerprint.

**Arquitectura general**

1. **Captura de paquetes**  
   Un sniffer o IDS (p. ej. Zeek, Suricata) inspecciona tráfico en tiempo real y, ante la detección de un payload sospechoso, extrae un _fingerprint_ (hash del payload, campos de cabecera, etc.).
2. **Pipeline de ingestión**  
   Cada fingerprint se envía a un componente de conteo de cardinalidad basado en HLL, que vive en memoria o en un datastore en memoria (p. ej. Redis con módulo HyperLogLog).
3. **Procesamiento por intervalos**  
   - Se crea un HLL con `b=14` (m=16 384 registros -> ~13 KB).  
   - Cada paquete malicioso se inserta con `hll.add(f)`.  
   - Al acabar el intervalo (p. ej. cada 60 s), se llama `n = hll.estimate()`.
4. **Generación de alertas**  
   - Si `n > umbral_absoluto` (p. ej. T = 1 000 hosts), dispara alerta crítica.  
   - Si `n / n_prev > umbral_relativo` (p. ej. >1.5× respecto al minuto anterior), alerta de crecimiento explosivo.
5. **Ventana deslizante**  
   Para suavizar picos, en vez de reiniciar, se mantienen k HLL circulares o se usan técnicas de "HLL decay" donde cada registro se reduce lentamente.


**Parámetros y precisión**

- **Elección de b=14**  
  - Memoria: $2^{14}$ registros → 16 384 enteros de 1 byte → ~16 KB.  
  - Error estándar $\sigma ≈ 1.04/\sqrt{m} \approx 1.04/128 \approx 0.8 %$.  
- **Umbrales**  
  - **Absoluto (T)**: depende de la capacidad de la red y del volumen normal de hosts clientes. Se calibra midiendo tráfico benigno.  
  - **Relativo**: detecta "brotes" que no alcanzarían T pero sí un rápido incremento.
- **Error en conteo**  
  - A menor cardinalidad (n < 2.5 m), se aplica linear counting y el sesgo es aún menor.  
  - A cardinalidades muy altas, se puede añadir corrección de rango alto.


**Ventanas temporales y "decay"**

1. **Ventanas fijas**  
   - Cada minuto se lanza un nuevo HLL; el anterior se descarta.  
   - Simple, pero no permite medias móviles.
2. **Ventana deslizante (sliding window)**  
   - Mantén un arreglo circular de `W` HLLs (p. ej. W = 5 para ventana de 5 min).  
   - Al insertarse un fingerprint, se añade a todos los HLLs activos.  
   - La estimación del total único en la ventana es la fusión de los W HLLs ($h_{total} = h_0 | h_1 | ... | h_{W-1}$).
3. **Decay exponencial**  
   - Cada registro $R_i$ se multiplica por un factor $\alpha <1$ al acabar cada intervalo, facilitando que los eventos antiguos "se desvanezcan".

**Integración con SIEM y alerta**

- Los valores `n` por intervalo se envían a la plataforma de monitoreo (Elastic, Splunk, Grafana).  
- Un job de alerta compara `n` contra umbrales históricos y disparadores definidos (por ejemplo, Elastic Watcher).  
- En dashboards, se grafica la serie temporal de estimaciones junto con la tasa de crecimiento minuto a minuto.


**Pseudocódigo simplificado**

```python
# Parámetros
b           = 14
intervalo_s = 60
T_absoluto  = 1000
T_relativo  = 1.5

# Buffers para ventana deslizante de 5 intervalos
W = 5
hlls = [HyperLogLog(b) for _ in range(W)]
i = 0

while True:
    t_inicio = time.time()
    hlls[i] = HyperLogLog(b)          # reset del HLL actual

    # Ingestión de paquetes durante intervalo
    for pkt in sniff_interval(intervalo_s):
        f = extract_fingerprint(pkt)
        for h in hlls:
            h.add(f)

    # Estimación de unión de ventana
    h_total = hlls[0]
    for j in range(1, W):
        h_total |= hlls[j]
    n = len(h_total)

    # Alerta absoluta
    if n > T_absoluto:
        alert(f"Hosts únicos infectados: {n} > {T_absoluto}")

    # Alerta relativa
    if i > 0:
        n_prev = medidas[i-1]
        if n_prev and (n / n_prev) > T_relativo:
            alert(f"Crecimiento rápido: {n_prev}→{n} (>×{T_relativo})")

    medidas[i] = n
    i = (i + 1) % W

    # Esperar fin de intervalo
    time.sleep(max(0, intervalo_s - (time.time() - t_inicio)))
```

#### **Agregación distribuida con HyperLogLog**

En un entorno distribuido, el reto de contar elementos únicos , ya sean usuarios activos, eventos distintos o claves de partición— puede convertirse en un cuello de botella si intentamos enviar todos los ítems brutos al nodo central. Aquí es donde brilla HyperLogLog: cada nodo procesa localmente su porción de datos y construye un pequeño "sketch" de cardinalidad, que luego se fusiona de forma trivial, sin necesidad de reenviar millones de registros.

Imaginemos un clúster de procesamiento de logs de acceso web. Cada servidor recibe decenas de miles de peticiones por segundo y necesita estimar cuántos visitantes únicos ha atendido. Con HyperLogLog, cada servidor mantiene un objeto `hll = HyperLogLog(b=14)` en memoria. A medida que llegan URLs, cabeceras o identificadores de sesión, el servidor hace:

```python
hll.add(user_id_hash)
```

Al finalizar el intervalo, por ejemplo, un minuto el sketch resultante ocupa apenas unos 16 KB ($2^{14}$ registros) y se envía al agregador central. Este, en lugar de recibir una lista de millones de hashes, recibe un array contiguo de enteros. La fusión es una operación $O(m)$ en la que para cada posición del vector se toma el máximo:

```python
hll_global = hll_shard1
for hll_shard in [hll_shard2, hll_shard3, ...]:
    hll_global |= hll_shard
```

El método `__or__` internamente invoca `merge()`, garantizando que `hll_global.estimate()` devuelva la cardinalidad aproximada total de todo el clúster.

**Ejemplo en Redis**

Redis incorpora este patrón de manera nativa con los comandos `PFADD`, `PFMERGE` y `PFCOUNT`. Cada instancia de tu aplicación puede hacer:

```redis
PFADD visits:shard1 user_1 user_2 user_3
PFADD visits:shard2 user_4 user_5 user_2
```

y luego, en el extremo central:

```redis
PFMERGE visits:global visits:shard1 visits:shard2
PFCOUNT visits:global
# → (integer) 5
```

Redis maneja la serialización del sketch y la fusión en segundo plano, de modo que tu único trabajo es emitir esos comandos.

**Integración en PostgreSQL y BigQuery**

En PostgreSQL, la extensión `hll` nos permite incluir HyperLogLog directamente en consultas SQL. Por ejemplo:

```sql
SELECT
  region,
  hll_cardinality(hll_union_agg(hll_hash_text(user_id))) AS usuarios_unicos
FROM access_logs
GROUP BY region;
```

Aquí cada fragmento del cluster ejecuta su agregación local, y PostgreSQL fusiona los sketches antes de calcular la cardinalidad.

De forma similar, en Google BigQuery basta con:

```sql
SELECT
  country,
  APPROX_COUNT_DISTINCT(user_id, 0.01) AS usuarios_unicos
FROM `dataset.events`
GROUP BY country;
```

BigQuery descompone el trabajo en shards, calcula un HLL por fragmento y fusiona todo automáticamente, todo de manera interna.


**Spark y Flink: conteos a escala de streaming**

En Apache Spark, la función de DataFrame `approx_count_distinct(col, rsd)` usa internamente HLL++:

```python
df.groupBy("session_id") \
  .agg(F.approx_count_distinct("user_id", rsd=0.01).alias("uniq_users"))
```

Spark mapea cada partición a un HLL, envía los sketches a los reducers y fusiona sin mover los datos individuales.

Análogamente, en Flink un `AggregateFunction` puede mantener un HLL por ventana de tiempo. Cuando se redistribuye el trabajo o reequilibra el paralelismo, Flink fusiona automáticamente los sketches de cada subtask usando el mismo operador de "máximo por bucket".

Con este modelo, HyperLogLog se convierte en la piedra angular de cualquier sistema que necesite contar alto volumen de ítems de forma eficiente, manteniendo un uso de memoria constante, operaciones de red ligeras y precisión suficiente (error típico <1 %) para toma de decisiones en tiempo real.

#### **Ejercicios**

**1. Estimación de frecuencia y elementos mayoritarios**

1. **Elemento mayoritario**  
   - Dado un flujo de $n$ elementos en el que sabes que existe uno que aparece más de $\lfloor n/2\rfloor$ veces, explica paso a paso cómo el algoritmo de Boyer–Moore encuentra ese elemento con espacio $O(1)$.  
   - Justifica por qué basta una sola pasada (más la verificación opcional) para garantizar la corrección.

2. **Heavy Hitters generales (Misra–Gries)**  
   - Para un umbral $\varphi$, razona por qué con $k=1/\varphi$ contadores se capturan todos los elementos cuya frecuencia supera $\varphi n$.  
   - Describe un escenario (por ejemplo, conteo de palabras en un log de servidor) y diseña cómo asignarías $\varphi$ para encontrar los "top-10" más frecuentes.


**2. Count–Min Sketch: funcionamiento interno**

1. **Fase de actualización**  
   - Ilustra con un ejemplo numérico (números pequeños) qué sucede en la matriz de contadores al insertar un mismo elemento tres veces seguidas.  
   - Compara el comportamiento de la **actualización normal** vs. la **conservadora** y discute en qué situaciones la segunda mejora la precisión.

2. **Fase de estimación**  
   - Explica por qué la estimación toma el **mínimo** de las $d$ celdas y no, por ejemplo, la media.  
   - Analiza cómo influyen las colisiones en cada celda y cómo el mínimo atenúa ese sesgo.

3. **Casos de uso**  
   - **Top-k restless sleepers**: propone un diseño de cómo mantiene el heap de tamaño $k$ junto al CMS, detallando la lógica de inserción y extracción de candidatos.  
   - **Similitud distribucional de palabras**: plantea cómo combinar dos CMS (marginal y conjunto) para estimar el PMI de un par de palabras.

4. **Error vs. espacio**  
   - A partir de los parámetros $\varepsilon$ y $\delta$, formula explícitamente $w$ y $d$.  
   - Para un flujo de $10^8$ eventos y un error relativo máximo del 0.5%, calcula $w$ y $d$ para $\delta=0.01$.  
   - Debate el impacto de doblar $w$ o doblar $d$ sobre el error y la memoria.

5. **Implementación conceptual**  
   - Describe con tus propias palabras la estructura de datos interna (matriz, semillas, total).  
   - Explica qué validaciones harías en el constructor para garantizar parámetros válidos y cómo documentarías la clase para un equipo de desarrollo.

6. **Intuición matemática**  
   - Deriva el límite de Markov que da origen a la probabilidad de colisión $\Pr[\text{error} \ge \varepsilon N]\le 1/e$.  
   - Extiende el razonamiento al caso de $d$ filas independientes y obtén la condición sobre $d$.

**3. Consultas por rangos con Count–Min Sketch**

1. **Intervalos diádicos**  
   - Explica por qué todo intervalo $[L,R]$ se puede cubrir con a lo sumo $2\log_2U$ bloques diádicos.  
   - Para $U=32$ y $[L,R]=[5,26]$, descompón manualmente en bloques de la forma $(i,j)$.

2. **Fase de actualización por niveles**  
   - Diseña el flujo de datos que llega (valores enteros) y detalla cómo actualizas cada CMS de nivel $i$.  
   - Reflexiona sobre el coste en tiempo y memoria cuando $U$ crece de $10^3$ a $10^6$.

3. **Fase de estimación de rango**  
   - Tras la descomposición diádica, argumenta cómo la suma de estimaciones mantiene la propiedad de cota superior.  
   - Discute cómo calibrar $\varepsilon$ para que el **error aditivo total** siga siendo aceptable en consultas largas.

4. **Cálculo de intervalos diádicos**  
   - Describe el algoritmo "greedy" paso a paso para encontrar el bloque más grande alineado con el inicio.  
   - Explica por qué la operación bitwise `L & -L` devuelve la mayor potencia de dos que divide a $L$.


**Estimación de cardinalidad y HyperLogLog**

1. **Conteo de elementos distintos**  
   - Compara el enfoque de **Flajolet–Martin** con un conteo exacto: ¿qué información almacena cada bit del registro?  
   - Justifica por qué basta con $O(\log\log N)$ bits para aproximar $\log N$.

2. **Diseño incremental de HyperLogLog**  
   - **Primer corte (probabilistic counting)**: explica la estadística de la posición del primer '1' en el hash y cómo se traduce a $\hat n = 2^R$.  
   - **Stochastic averaging**: describe el "truco de los limones" (varios registros y promedio) y por qué reduce la varianza.  
   - **LogLog vs. HyperLogLog**: analiza la diferencia clave entre usar promedio aritmético y promedio armónico.

3. **Experimento práctico**  
   - Propón un diseño de mini-experimento variando el número de buckets $m$, escribe los pasos de evaluación (sin código) y qué métricas registrarías (error medio, desviación).

4. **Caso de uso: Catching worms**  
   - Imagina un sistema de detección de gusanos en logs de red: define qué representa cada elemento y cómo interpreta el HLL para alertar a operaciones.

5. **Agregación distribuida con HLL**  
   - Detalla cómo fusionar múltiples estructuras HLL de diferentes nodos y por qué la operación de "máximo por bucket" mantiene la corrección.  
   - Discute el coste de comunicación y memoria en comparación con un conteo de cardinalidad exacto.


In [ ]:
## Tus respuestas